# Neptune + Keras

## Introduction

This guide will show you how to:

* Initialize Neptune and create a `run`,
* Create a `NeptuneCallback()`,
* Log model training metrics to Neptune using `NeptuneCallback()`
* Log additional metadata like hyperaparameters, images, and model weights to Neptune.

[See this example in Neptune](https://app.neptune.ai/o/common/org/tf-keras-integration/e/TFK-40889/all)

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install -U neptune-client neptune-tensorflow-keras tensorflow

## Import libraries

In [ ]:
import tensorflow as tf

## Define your model, data loaders, and optimizer

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.keras.activations.relu),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax),
    ]
)

optimizer = tf.keras.optimizers.SGD(
    lr=0.005,
    momentum=0.4,
)

model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

## Quickstart

### Start a run

To connect your script to Neptune and create a new run, we tell Neptune:
* **Who you are** - with a Neptune API token
* **Where to send your data** - to a Neptune project

The cell below lets you record data to the public project [common/tf-keras-integration](https://app.neptune.ai/o/common/org/tf-keras-integration) as an anonymous user.

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init_run(project="common/tf-keras-integration", api_token=neptune.ANONYMOUS_API_TOKEN)

Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next. 

Alternatively, you can log the example to your own workspace.

To do that, replace the code above with the following:

```python
from getpass import getpass

run = neptune.init_run(
    api_token=getpass("Enter your Neptune API token: "),
    project="workspace-name/project-name",  # replace with your own
)
```

For example, if your workspace name is `ml-team` and the project name is `classification`, the project argument is: `project="ml-team/classification"`.

To find your API token and project name, [log in to Neptune](https://app.neptune.ai/).
- In the top-right corner, click your avatar and select **Get your API token**.
- To find and copy your project name, navigate to the project, then click **Settings** → **Properties**.

### Add NeptuneCallback() to model.fit()

In [ ]:
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

neptune_cbk = NeptuneCallback(run=run)

model.fit(x_train, y_train, epochs=5, batch_size=64, callbacks=[neptune_cbk])

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

### Explore results in the Neptune app

Go to the run link and explore charts that were created from the metric values you logged. 

## More Options

You can log way more than just metrics to Neptune. 

Let's go over some of those options here. 

### Log hyperparameters

If you want to log hyperparameters of your model training run you just need to pass them to the `base_namespace` of your choice.

In [ ]:
run_2 = neptune.init_run(
    project="common/tf-keras-integration", api_token=neptune.ANONYMOUS_API_TOKEN
)

params = {"lr": 0.005, "momentum": 0.9, "epochs": 10, "batch_size": 32}

# log hyper-parameters
run_2["hyper-parameters"] = params

optimizer = tf.keras.optimizers.SGD(learning_rate=params["lr"], momentum=params["momentum"])

model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

neptune_cbk_2 = NeptuneCallback(run=run_2)  # base_namespace defaults to "training"

model.fit(
    x_train,
    y_train,
    epochs=params["epochs"],
    batch_size=params["batch_size"],
    callbacks=[neptune_cbk_2],
)

Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next. 

### Log test sample images

In [ ]:
from neptune.new.types import File

for image, label in zip(x_test[:10], y_test[:10]):
    prediction = model.predict(image[None], verbose=0)
    predicted = prediction.argmax()
    desc = f"label : {label} | predicted : {predicted}"
    run_2["visualization/test_prediction"].log(File.as_image(image), description=desc)

### Log model weights

In [ ]:
model.save("my_model")

run_2["model_checkpoint/checkpoint"].upload_files("my_model/*")

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run_2.stop()

## Explore results in the Neptune app

You can also check out an [example run](https://app.neptune.ai/o/common/org/tf-keras-integration/e/TFK-40889/all)